# Chain of Thought Prompting
Chain-of-Thought (CoT) Prompting is a technique that enhances the reasoning capabilities of large language models (LLMs) by incorporating logical steps—or a “chain of thought”—within the prompt. Unlike direct-answer prompting, CoT guides the model to work through intermediate reasoning steps, making it more adept at solving complex tasks like math problems, commonsense reasoning, and symbolic manipulation.

## How it differs from traditional prompting techniques
- **Direct Answer Prompting**: Traditional prompting often leads to direct answers without intermediate reasoning. This can result in incorrect or incomplete responses, especially for complex tasks because the model may not have enough context to understand the problem fully and/or don't have the necessary knowledge to reason through the problem.
- **Intermediate Steps**: CoT prompting encourages the model to break down the problem into smaller, manageable steps, rather than providing a direct answer. It is essential for encouraging the model to perform multi-step reasoning, which is crucial for tasks that require logical deduction or complex calculations.
- **Achieving Efficiency without Finetuning by embedding reasoning in the prompt**: CoT works across tasks without the need for fine-tuning, using a standard prompt format that embeds reasoning, thus simplifying adaptation to various complex tasks.

## How Chain-of-Thought Prompting Works
1. **Decompose the Problem**: CoT prompts guide the model to break down a complex question into manageable steps, akin to how a human might solve the problem.
2. **Guide with Exemplars**: CoT uses examples that demonstrate reasoning steps, helping the model grasp the method needed to reach the correct answer.

## Example

First we try a simple example without any reasoning steps. Here we are trying to count the number of "I"s in the word "Antidisestablishmentarianism".

In [12]:
from ollama import generate, GenerateResponse
from typing import Iterator

prompt = """
How many “I”s are in the word “Antidisestablishmentarianism”?
"""

stream: Iterator[GenerateResponse] = generate(
    model="gemma3:12b",
    prompt=prompt,
    stream=True
)
for chunk in stream:
    print(chunk.response, end="", flush=True)

Let's count them!

The word "Antidisestablishmentarianism" has **four** "I"s.

As we see, this answer is wrong. Now lets try to add some examples for a traditional one shot prompt without reasoning.

In [15]:
from ollama import generate, GenerateResponse
from typing import Iterator

prompt = """
Question: How many “R”s are in the word “strawberry”?
Answer: 3
Question: How many “I”s are in the word “Antidisestablishmentarianism”?
Answer:
"""

stream: Iterator[GenerateResponse] = generate(
    model="gemma3:12b",
    prompt=prompt,
    stream=True
)
for chunk in stream:
    print(chunk.response, end="", flush=True)

Answer: 3

As we see, the answer is still wrong with the one shot example. Now lets try to add chain of thought by embedding the reasoning in the example.

In [14]:
from ollama import generate, GenerateResponse
from typing import Iterator

prompt = """
Question: How many “R”s are in the word “strawberry”?
Answer: Lets think step by step. we break the word "strawberry" into its letters. The letters are s, t, r, a, w, b, e, r, r, y. Now we iterate through the letters and count the number of "R"s. at the beginning we set the counter to 0 and we increment it as soon as we come across any 'R' letter.
start iteration:
1. letter 1: 's' -> it is not 'r' so we don't increase the counter. counter -> 0
2. letter 2: 't' -> it is not 'r' so we don't increase the counter. counter -> 0
3. letter 3: 'r' -> it is 'r' so we increase the counter. counter -> 1
4. letter 4: 'a' -> it is not 'r' so we don't increase the counter. counter -> 1
5. letter 5: 'w' -> it is not 'r' so we don't increase the counter. counter -> 1
6. letter 6: 'b' -> it is not 'r' so we don't increase the counter. counter -> 1
7. letter 7: 'e' -> it is not 'r' so we don't increase the counter. counter -> 1
8. letter 8: 'r' -> it is 'r' so we increase the counter. counter -> 2
9. letter 9: 'r' -> it is 'r' so we increase the counter. counter -> 3
10. letter 10: 'y' -> it is not 'r' so we don't increase the counter. counter -> 3
Now we have completed the iteration through the letters of the word "strawberry". The final value of the counter is 3. So the answer is 3.

Question: How many “I”s are in the word “Antidisestablishmentarianism”?
Answer: Lets think step by step.
"""

stream: Iterator[GenerateResponse] = generate(
    model="gemma3:12b",
    prompt=prompt,
    stream=True
)
for chunk in stream:
    print(chunk.response, end="", flush=True)

Let's think step by step. We break the word "Antidisestablishmentarianism" into its letters. The letters are A, n, t, i, d, i, s, e, s, t, a, b, l, i, s, h, m, e, n, t, a, r, i, a, n, i, s, m. Now we iterate through the letters and count the number of "I"s.

start iteration:
1. letter 1: 'A' -> it is not 'i' so we don't increase the counter. counter -> 0
2. letter 2: 'n' -> it is not 'i' so we don't increase the counter. counter -> 0
3. letter 3: 't' -> it is not 'i' so we don't increase the counter. counter -> 0
4. letter 4: 'i' -> it is 'i' so we increase the counter. counter -> 1
5. letter 5: 'd' -> it is not 'i' so we don't increase the counter. counter -> 1
6. letter 6: 'i' -> it is 'i' so we increase the counter. counter -> 2
7. letter 7: 's' -> it is not 'i' so we don't increase the counter. counter -> 2
8. letter 8: 'e' -> it is not 'i' so we don't increase the counter. counter -> 2
9. letter 9: 's' -> it is not 'i' so we don't increase the counter. counter -> 2
10. letter 10: 

Now we see that the model broke down the problem into multiple steps and provided the correct answer. This is the essence of chain of thought prompting.